In [12]:
!pip install pyautogen

In [13]:
!pip install groq

In [14]:
from google.colab import userdata
api_key = userdata.get('groq_api_key')

In [15]:
import autogen
from autogen import UserProxyAgent
from autogen import AssistantAgent


config_list = [{'model': 'llama3-8b-8192',
                "api_type": "groq",
                "api_key": api_key,
             }]

llama3_8b_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

**User Proxy Agent:** Human input to the user agent. Business requirement provided to the proxy agent. Initiate chat between architects

**Assistant Agent**




In [16]:
task = '''
 **Task**: As an architect, you are required to design a solution for the
 following business requirements:
    - Data storage for massive amounts of IoT data
    - Real-time data analytics and machine learning pipeline
    - Scalability
    - Cost Optimization
    - Region pairs in Europe, for disaster recovery
    - Tools for monitoring and observability
    - Timeline: 6 months

    Break down the problem using a Chain-of-Thought approach. Ensure that your
    solution architecture is following best practices.
    '''

**Cloud Prompt**

In [17]:
cloud_prompt = '''
**Role**: You are an expert cloud architect. You need to develop architecture proposals
using either cloud-specific PaaS services, or cloud-agnostic ones.
The final proposal should consider all 3 main cloud providers: Azure, AWS and GCP, and provide
a data architecture for each. At the end, briefly state the advantages of cloud over on-premises
architectures, and summarize your solutions for each cloud provider using a table for clarity.
'''
cloud_prompt += task

**OSS Architect**

In [18]:
oss_prompt = '''
**Role**: You are an expert on-premises, open-source software architect. You need
to develop architecture proposals without considering cloud solutions.
 Only use open-source frameworks that are popular and have lots of active contributors.
 At the end, briefly state the advantages of open-source adoption, and summarize your
 solutions using a table for clarity.
'''
oss_prompt += task

**Lead Architect**

In [19]:
lead_prompt =  '''
**Role**: You are a lead Architect tasked with managing a conversation between
the cloud and the open-source Architects.
Each Architect will perform a task and respond with their resuls. You will critically
review those and also ask for, or point to, the disadvantages of their solutions.
You will review each result, and choose the best solution in accordance with the business
requirements and architecture best practices. You will use any number of summary tables to
communicate your decision.
'''
lead_prompt += task

In [20]:
user_proxy = autogen.UserProxyAgent(
    name="supervisor",
    system_message = "A Human Head of Architecture",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

cloud_agent = AssistantAgent(
    name = "cloud",
    system_message = cloud_prompt,
    llm_config={"config_list": config_list}
    )

oss_agent = AssistantAgent(
    name = "oss",
    system_message = oss_prompt,
    llm_config={"config_list": config_list}
    )

lead_agent = AssistantAgent(
    name = "lead",
    system_message = lead_prompt,
    llm_config={"config_list": config_list}
)

In [21]:
def state_transition(last_speaker, groupchat):
   messages = groupchat.messages
   if last_speaker is user_proxy:
       return cloud_agent
   elif last_speaker is cloud_agent:
       return oss_agent
   elif last_speaker is oss_agent:
       return lead_agent
   elif last_speaker is lead_agent:
       # lead -> end
       return None

In [22]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, cloud_agent, oss_agent, lead_agent],
    messages=[],
    max_round=6,
    speaker_selection_method=state_transition,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llama3_8b_config)

user_proxy.initiate_chat(
    manager, message="Provide your best architecture based on these business requirements."
)

supervisor (to chat_manager):

Provide your best architecture based on these business requirements.

--------------------------------------------------------------------------------

Next speaker: cloud



APIConnectionError: Connection error.